<a href="https://colab.research.google.com/github/AIandAutomationTools/ai-quartet-evaluator/blob/main/ai_quartet_cleaned_pitch_analysis_version25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🎵 AI Quartet Pitch Analysis
This notebook compares a student's pitch to a professor's using a Google Drive MP3 link for each.

In [3]:
# === Imports ===
import librosa
import numpy as np
import matplotlib.pyplot as plt
import requests
import io
import os
import urllib.parse
from google.colab import output
import time

# Initialize
student_url = None
professor_url = None

# Temporary override for debugging
#student_url = "https://drive.google.com/uc?export=download&id=15MRshVMuFLa5Jzrjk6DiL-R1t6u-msjK"
#professor_url = "https://drive.google.com/uc?export=download&id=19GFc2yNb-voEjsczGtxcK_No4XGDS7Wp"

#print("✅ Student URL:", student_url)
#print("✅ Professor URL:", professor_url)



# Set function
def set_urls(student, professor):
    global student_url, professor_url
    student_url = urllib.parse.unquote(student)
    professor_url = urllib.parse.unquote(professor)
    print("✅ URLs successfully received in Python callback!")
    print("Student URL (raw):", student_url)
    print("Professor URL (raw):", professor_url)

# Register function
output.register_callback('set_urls', set_urls)

# Run JS
output.eval_js('''
(async () => {
  const params = new URLSearchParams(window.location.search);
  const student = params.get("student_url");
  const professor = params.get("professor_url");
  if (student && professor) {
    console.log("✅ Found URL parameters:", student, professor);
    await google.colab.kernel.invokeFunction('set_urls', [student, professor], {});
  } else {
    console.warn("❌ Missing one or both URL parameters in query string.");
  }
})();
''')


✅ Student URL: https://drive.google.com/uc?export=download&id=15MRshVMuFLa5Jzrjk6DiL-R1t6u-msjK
✅ Professor URL: https://drive.google.com/uc?export=download&id=19GFc2yNb-voEjsczGtxcK_No4XGDS7Wp


In [4]:
# === Cell 2: Wait + Validate ===
# Wait for callback to set URLs
for _ in range(20):  # Up to 10 seconds
    if student_url and professor_url:
        break
    time.sleep(0.5)

# Final validation
if not student_url or not professor_url:
    raise ValueError("❌ Missing one or both URLs. Please ensure they are passed in the notebook link.")

print("🎯 Student URL:", student_url)
print("🎯 Professor URL:", professor_url)


🎯 Student URL: https://drive.google.com/uc?export=download&id=15MRshVMuFLa5Jzrjk6DiL-R1t6u-msjK
🎯 Professor URL: https://drive.google.com/uc?export=download&id=19GFc2yNb-voEjsczGtxcK_No4XGDS7Wp


In [5]:
def load_audio_from_url(url):
    if not url:
        raise ValueError("❌ Missing URL input.")
    try:
        print(f"🔗 Attempting to download: {url}")
        response = requests.get(url)
        response.raise_for_status()
        audio_data = io.BytesIO(response.content)
        audio, sr = librosa.load(audio_data, sr=None)
        return audio, sr
    except Exception as e:
        raise RuntimeError(f"❌ Failed to load audio from {url}\nError: {e}")

In [6]:

student_url = ""
professor_url = ""

def set_urls(student, professor):
    global student_url, professor_url
    student_url = urllib.parse.unquote(student)
    professor_url = urllib.parse.unquote(professor)

output.register_callback('set_urls', set_urls)


# Now you can call it
params = get_url_params()
student_url = params.get("student_url")
professor_url = params.get("professor_url")

#print("Professor URL:", professor_url)
#print("Student URL:", student_url)

import time
time.sleep(1)  # Allow time for JS to inject values

print("🎧 Student URL:", student_url)
print("🎓 Professor URL:", professor_url)

import time

# Wait up to 5 seconds for the URLs to populate
for _ in range(10):
    if student_url and professor_url:
        break
    time.sleep(0.5)

# Now check

if student_url is None or professor_url is None:
    raise ValueError("❌ Missing one or both URLs. Please ensure they are passed in the notebook link.")




NameError: name 'get_url_params' is not defined

In [ ]:
import time

# Wait up to 5 seconds for values to be set
for _ in range(10):
    if student_url and professor_url:
        break
    time.sleep(0.5)

# Check if values are present
if not student_url or not professor_url:
    raise ValueError("❌ Missing one or both URLs. Please ensure they are passed in the notebook link.")
else:
    print("✅ URLs successfully loaded.")
    print("Professor URL:", professor_url)
    print("Student URL:", student_url)


prof_audio, sr1 = load_audio_from_url(professor_url)
stud_audio, sr2 = load_audio_from_url(student_url)

# Resample if needed
if sr1 != sr2:
    stud_audio = librosa.resample(stud_audio, orig_sr=sr2, target_sr=sr1)
    sr2 = sr1

In [ ]:
# Use yin for monophonic pitch detection
prof_pitch = librosa.yin(prof_audio, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'), sr=sr1)
stud_pitch = librosa.yin(stud_audio, fmin=librosa.note_to_hz('C2'), fmax=librosa.note_to_hz('C7'), sr=sr2)

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(prof_pitch, label='Professor Pitch', alpha=0.75)
plt.plot(stud_pitch, label='Student Pitch', alpha=0.75)
plt.legend()
plt.title("Pitch Comparison")
plt.xlabel("Frame")
plt.ylabel("Pitch (Hz)")
plt.grid()
plt.show()

In [ ]:
prof_pitch_clean = np.nan_to_num(prof_pitch)
stud_pitch_clean = np.nan_to_num(stud_pitch)

# Ensure both pitch arrays are the same length
min_len = min(len(prof_pitch_clean), len(stud_pitch_clean))
prof_pitch_clean = prof_pitch_clean[:min_len]
stud_pitch_clean = stud_pitch_clean[:min_len]

# Now compute pitch difference
pitch_diff = np.abs(prof_pitch_clean - stud_pitch_clean)

pitch_diff = np.abs(prof_pitch_clean - stud_pitch_clean)
avg_error = np.mean(pitch_diff)

# Analyze pitch deviations over time
time_per_frame = len(stud_audio) / sr2 / len(stud_pitch_clean)
significant_errors = []

for i, diff in enumerate(pitch_diff):
    if diff > 50:  # Threshold for "significant" error
        timestamp = round(i * time_per_frame, 1)
        significant_errors.append(timestamp)

# Group timestamps into continuous segments
from itertools import groupby
from operator import itemgetter

segments = []
for k, g in groupby(enumerate(significant_errors), lambda x: x[0] - x[1]):
    group = list(map(itemgetter(1), g))
    if group:
        start = group[0]
        end = group[-1]
        if end - start >= 1.5:  # Only include segments > 1.5 seconds
            segments.append((start, end))

# Provide localized feedback
print("\n🎵 Average pitch difference: {:.2f} Hz".format(avg_error))

if segments:
    print("⚠️ Detailed pitch feedback (segments with >1.5s deviation):")
    for start, end in segments:
        print(f" - Pitch off between {start:.1f}s and {end:.1f}s")
elif avg_error > 100:
    print("⚠️ Pitch is consistently off — not just in isolated areas. Focus on tuning accuracy throughout the piece.")
else:
    print("✅ No major localized pitch issues detected.")


#print(f"\n🎵 Average pitch difference: {avg_error:.2f} Hz")
#if avg_error < 20:
  #  print("🎯 Great job! Your pitch closely matches the reference.")
#elif avg_error < 50:
    print("👍 You're in the ballpark, but there’s room for improvement.")
#else:
 #   print("⚠️ Your pitch deviates significantly. Focus on tuning and accuracy.")


